In [1]:
import pandas as pd


In [18]:
df = pd.read_csv('jobOffers2.csv', index_col=0)

In [19]:
df.head()


,jobTitles,company,salaryRange,Bonus,category+progLang,seniority,workLanguage,location,remote
0,Django backend Developer,RES-Q Global,55k - 80k CZK,NaN,"Kategorie: Backend , python","Mid, Senior","Pracovní jazyk:Angličtina, Čeština","Brno, Božetěchova 62 •Brno, Božetěchova 62",NaN
1,Team Lead Backend Development (m/f/d),PlanRadar,125.9k - 188.9k CZK,NaN,"Kategorie: Backend , ruby on rails",Expert,Pracovní jazyk:Angličtina,"Hybrid +1 •Vienna, Kärntner Ring 5-7",NaN
2,NodeJS Backend Developer 🔥,Dayswaps,67.2k - 117.6k CZK,10k CZK,"Kategorie: Backend , node",Mid,"Pracovní jazyk:Čeština, Slovenština","Hybrid +1 •Prague, Na Zderaze 5",NaN
3,Backend Developer,"PECOSTA, a.s.",60k - 90k CZK,NaN,"Kategorie: Backend , .net","Mid, Senior",Pracovní jazyk:Čeština,"Hybrid +1 •Ostrava, Nemocniční 987/12",NaN
4,Backend Developer,wpj s.r.o.,40k - 70k CZK,NaN,"Kategorie: Backend , php","Mid, Senior",Pracovní jazyk:Čeština,"Hybrid +1 •Vrchlabí, Lánovská 1475",NaN


In [20]:
#salary
#dropping the non-paid offers
df = df.drop(df[df['salaryRange'] == 'neplacená stáž'].index)

#getting lower and upper ranges of the possible salary
df['salLower'] = df['salaryRange'].str.split(" - ").str[0].str.split("  ").str[0]
df['salUpper'] = df['salaryRange'].str.split(" - ").str[1].str.split("  ").str[0]

df['salLower'] = df['salLower'].replace({'k': '*1e3'}, regex=True).map(pd.eval).astype(int)
df['salUpper'].fillna(df['salLower'], inplace = True)
df['salUpper'] = df['salUpper'].replace({'k': '*1e3'}, regex=True).map(pd.eval).astype(int)

#getting the average of the salary range
df['salAvg'] = (df['salUpper'] + df['salLower'])/2


In [37]:
df['category+progLang'] = df['category+progLang'].str.replace("Kategorie:","").str.strip()

In [38]:
#category and main programming language

df['category'] = df['category+progLang'].str.split(",").str[0]
df['mainProgLang'] = df['category+progLang'].str.split(",").str[1]
df.head()

,jobTitles,company,salaryRange,Bonus,category+progLang,seniority,workLanguage,location,remote,salLower,...,salAvg,category,mainProgLang,seniorityAdj,language,angličtina,česky,mainProgLangAdj,remoteBool,BonusAdj
0,Django backend Developer,RES-Q Global,55k - 80k CZK,NaN,"Backend , python","Mid, Senior","Pracovní jazyk:Angličtina, Čeština","Brno, Božetěchova 62 •Brno, Božetěchova 62",No remote,55000,...,67500.0,Backend,python,Mid Advanced,"[Pracovní jazyk:Angličtina, Čeština]",False,False,NaN,False,No bonus
1,Team Lead Backend Development (m/f/d),PlanRadar,125.9k - 188.9k CZK,NaN,"Backend , ruby on rails",Expert,Pracovní jazyk:Angličtina,"Hybrid +1 •Vienna, Kärntner Ring 5-7",No remote,125900,...,157400.0,Backend,ruby on rails,Senior,[Pracovní jazyk:Angličtina],False,False,NaN,False,No bonus
2,NodeJS Backend Developer 🔥,Dayswaps,67.2k - 117.6k CZK,10k CZK,"Backend , node",Mid,"Pracovní jazyk:Čeština, Slovenština","Hybrid +1 •Prague, Na Zderaze 5",No remote,67200,...,92400.0,Backend,node,Mid,"[Pracovní jazyk:Čeština, Slovenština]",False,False,NaN,False,No bonus
3,Backend Developer,"PECOSTA, a.s.",60k - 90k CZK,NaN,"Backend , .net","Mid, Senior",Pracovní jazyk:Čeština,"Hybrid +1 •Ostrava, Nemocniční 987/12",No remote,60000,...,75000.0,Backend,.net,Mid Advanced,[Pracovní jazyk:Čeština],False,False,NaN,False,No bonus
4,Backend Developer,wpj s.r.o.,40k - 70k CZK,NaN,"Backend , php","Mid, Senior",Pracovní jazyk:Čeština,"Hybrid +1 •Vrchlabí, Lánovská 1475",No remote,40000,...,55000.0,Backend,php,Mid Advanced,[Pracovní jazyk:Čeština],False,False,NaN,False,No bonus


In [23]:
# adjusting the names of the seniority column
df["seniorityAdj"] = df["seniority"].str.strip().replace({'Trainee, Junior': 'Junior', 'Trainee': 'Junior','Senior, Expert':'Senior',
                                                          'Expert':'Senior', 'Mid, Senior':'Mid Advanced'})
df.groupby(['seniorityAdj']).size()   
  

seniorityAdj
Mid             16
Mid Advanced    30
Senior          25
dtype: int64

In [24]:
#Work language
df["workLanguage"]=df["workLanguage"].str.replace("Work language:","")
df['language'] = df['workLanguage'].str.split(',')

df.groupby(['workLanguage']).size()
langList = ['angličtina','česky']
for lang in langList:
    df[lang] = df['workLanguage'].str.contains(lang)

#['angličtina', 'česky' ]


In [25]:
#converting less demanded programming languages to 'other'
mask = df['mainProgLang'].map(df['mainProgLang'].value_counts())<20
df['mainProgLangAdj']=df['mainProgLang'].mask(mask,'other')
counts = df.groupby(['mainProgLangAdj']).size().sort_values(ascending=False)
print(counts)

Series([], dtype: int64)


In [26]:
#df['location'].str.contains('Praha|Prague')
df['remoteBool'] = df['remote'].notnull().astype('boolean')
df.loc[df['remote'].isna(), 'remote'] = 'No remote'

In [27]:
#adjusting bonus variable, few obsevations have included the bonus in absolute term e.g. 2000CZK instead in the percentages
mask = df['Bonus'].str.contains('%', na = False)==False
df['BonusAdj']=df['Bonus'].mask(mask,'No bonus').str.strip()
df['BonusAdj'].unique()


array(['No bonus', '+15%', '+30%', '+75%'], dtype=object)

In [28]:
cleanDF = df[['jobTitles', 'company', 'BonusAdj', 'salLower', 'salUpper','salAvg', 'category', 'mainProgLangAdj', 'seniorityAdj', 'angličtina', 'česky', 'remoteBool']]

In [29]:
# drop all job offers, that only require some foreign language outside Czech or English
cleanDF = cleanDF.loc[cleanDF.angličtina | cleanDF.česky ]


In [30]:
cleanDF.head()

,jobTitles,company,BonusAdj,salLower,salUpper,salAvg,category,mainProgLangAdj,seniorityAdj,angličtina,česky,remoteBool


In [16]:
df.head()

,jobTitles,company,salaryRange,Bonus,category+progLang,seniority,workLanguage,location,remote,salLower,...,salAvg,category,mainProgLang,seniorityAdj,language,angličtina,česky,mainProgLangAdj,remoteBool,BonusAdj
0,Java Developer,eppTec s.r.o.,105k - 168k CZK,NaN,"Backend, java","Mid, Senior",česky,"Hybrid +1 • Praha 40% on-site, 60% remotely",No remote,105000,...,136500.0,Backend,java,Mid Advanced,[česky],False,True,java,False,No bonus
1,SW Application Developer C++,Consultech Recruitment Services s.r.o,60k - 95k CZK,NaN,"Backend, MS Visual Studio",Mid,,"Hybrid +1 • Prague 80% on-site, 20% remotely",No remote,60000,...,77500.0,Backend,MS Visual Studio,Mid,[],False,False,other,False,No bonus
2,Junior C++ Developer,Consultech Recruitment Services s.r.o,45k - 55k CZK,NaN,"Backend, C++",Junior,,"Hybrid +1 • Plzen 80% on-site, 20% remotely",No remote,45000,...,50000.0,Backend,C++,Junior,[],False,False,other,False,No bonus
3,Python / Delphi Developer,"Kolibrik.net, sro",40k - 55k CZK,NaN,"Backend, python",Mid,česky,"Hybrid +1 • Žďár nad Sázavou , Smetanova 116...",No remote,40000,...,47500.0,Backend,python,Mid,[česky],False,True,python,False,No bonus
4,3d Graphics Developer,Sourcein,63k - 115.5k CZK,NaN,"Backend, c++",Mid,česky,Prague (Po pandemi) • Prague,Covid-time na dálku,63000,...,89250.0,Backend,c++,Mid,[česky],False,True,other,True,No bonus


In [126]:
cleanDF.to_csv('jobOffersClean.csv')